In [3]:
#This code is from download_genomes_Entrez in /jupyter

#Download all hanta genomes, all segments


from Bio import Entrez
from Bio import SeqIO
Entrez.api_key = "<enter_your_api_key_here>"
import pandas as pd
import re

Entrez.email = "<enter_your_email_here>"  # Always tell NCBI who you are
handle = Entrez.esearch(db="nucleotide", term="Sin Nombre orthohantavirus[Primary Organism]", idtype="acc", usehistory="y", retmax=10000)
record= Entrez.read(handle)

webenv = record["WebEnv"]
query_key = record["QueryKey"]


count = record["Count"]
count=int(count)
idlist = ", ".join(record["IdList"][:count])
print(count)

#Download xml files from Genbank
batch_size = 20

all_data=[]

for start in range(0, count, batch_size):
    end = min(count, start + batch_size)
    print("Going to download record %i to %i" % (start + 1, end))
    fetch_handle = Entrez.efetch(
        db="nucleotide",
        retmode="xml",
        id=idlist,
        retstart=start,
        retmax=batch_size,
        webenv=webenv,
    )
    datum=Entrez.read(fetch_handle)
    all_data.extend(datum)

fetch_handle.close()

print(len(all_data))

df_columns=['accession_num', 'strain', 'isolate', 'segment', 'chromosome', 'sequence_length', 'host', 'location', 'subtype', 'organism', 'collection_date', 'sequence']
sigma=15
df_Compiled_Data = pd.DataFrame(columns=df_columns)

#all_data is a list that's compiled from the esearch function results in cell #163.
#for loop below iterates through all of the returned search results.
for gb_entry in range(0, len(all_data)):
    #print(datum[gb_entry])
    accession_number=""
    strain=""
    isolate=""
    segment=""
    chromosome=""
    sequence_length=""
    host=""
    location=""
    subtype=""
    organism=""
    date=""
    sequence=""
    
    #Grab accession number.
    #print(all_data[gb_entry]["GBSeq_locus"])
    accession_number=all_data[gb_entry]["GBSeq_locus"]
    
    #Grab sequence.
    sequence=all_data[gb_entry]["GBSeq_sequence"]
    #print(sequence)
    
    #Iterate through all of the GBFeature_quals and process the data:
    for item in range(0, (len(all_data[gb_entry]["GBSeq_feature-table"][0]["GBFeature_quals"]))):
        if(all_data[gb_entry]["GBSeq_feature-table"][0]["GBFeature_quals"][item]["GBQualifier_name"] == "strain"):
            #print(all_data[gb_entry]["GBSeq_feature-table"][0]["GBFeature_quals"][item]["GBQualifier_value"])
            strain=all_data[gb_entry]["GBSeq_feature-table"][0]["GBFeature_quals"][item]["GBQualifier_value"]
            
        if(all_data[gb_entry]["GBSeq_feature-table"][0]["GBFeature_quals"][item]["GBQualifier_name"] == "isolate"):
            #print(all_data[gb_entry]["GBSeq_feature-table"][0]["GBFeature_quals"][item]["GBQualifier_value"])
            isolate=all_data[gb_entry]["GBSeq_feature-table"][0]["GBFeature_quals"][item]["GBQualifier_value"]
            
        if(all_data[gb_entry]["GBSeq_feature-table"][0]["GBFeature_quals"][item]["GBQualifier_name"] == "segment"):
            #print(all_data[gb_entry]["GBSeq_feature-table"][0]["GBFeature_quals"][item]["GBQualifier_value"])
            segment=all_data[gb_entry]["GBSeq_feature-table"][0]["GBFeature_quals"][item]["GBQualifier_value"]
            
        if(all_data[gb_entry]["GBSeq_feature-table"][0]["GBFeature_quals"][item]["GBQualifier_name"] == "chromosome"):
            #print(all_data[gb_entry]["GBSeq_feature-table"][0]["GBFeature_quals"][item]["GBQualifier_value"])
            chromosome=all_data[gb_entry]["GBSeq_feature-table"][0]["GBFeature_quals"][item]["GBQualifier_value"]
            
        if(all_data[gb_entry]["GBSeq_feature-table"][0]["GBFeature_quals"][item]["GBQualifier_name"] == "host"):
            #print(all_data[gb_entry]["GBSeq_feature-table"][0]["GBFeature_quals"][item]["GBQualifier_value"])
            host=all_data[gb_entry]["GBSeq_feature-table"][0]["GBFeature_quals"][item]["GBQualifier_value"]
        
        if(all_data[gb_entry]["GBSeq_feature-table"][0]["GBFeature_quals"][item]["GBQualifier_name"] == "country"):
            #print(all_data[gb_entry]["GBSeq_feature-table"][0]["GBFeature_quals"][item]["GBQualifier_value"])
            location=all_data[gb_entry]["GBSeq_feature-table"][0]["GBFeature_quals"][item]["GBQualifier_value"]
            
        if(all_data[gb_entry]["GBSeq_feature-table"][0]["GBFeature_quals"][item]["GBQualifier_name"] == "subtype"):
            #print(all_data[gb_entry]["GBSeq_feature-table"][0]["GBFeature_quals"][item]["GBQualifier_value"])
            subtype=all_data[gb_entry]["GBSeq_feature-table"][0]["GBFeature_quals"][item]["GBQualifier_value"]
            
        if(all_data[gb_entry]["GBSeq_feature-table"][0]["GBFeature_quals"][item]["GBQualifier_name"] == "organism"):
            #print(all_data[gb_entry]["GBSeq_feature-table"][0]["GBFeature_quals"][item]["GBQualifier_value"])
            organism=all_data[gb_entry]["GBSeq_feature-table"][0]["GBFeature_quals"][item]["GBQualifier_value"]
            
        #if(all_data[gb_entry]["GBSeq_feature-table"][0]["GBFeature_quals"][item]["GBQualifier_name"] == "collection_date"):
        if(re.search("collection_date", all_data[gb_entry]["GBSeq_feature-table"][0]["GBFeature_quals"][item]["GBQualifier_name"], re.IGNORECASE) is not None):
            #print(all_data[gb_entry]["GBSeq_feature-table"][0]["GBFeature_quals"][item]["GBQualifier_value"])
            date=all_data[gb_entry]["GBSeq_feature-table"][0]["GBFeature_quals"][item]["GBQualifier_value"]
        
        #if(all_data[gb_entry]["GBSeq_feature-table"][0]["GBFeature_quals"][item]["GBQualifier_name"] == "date"):
            #print(all_data[gb_entry]["GBSeq_feature-table"][0]["GBFeature_quals"][item]["GBQualifier_value"])
        #   date=all_data[gb_entry]["GBSeq_feature-table"][0]["GBFeature_quals"][item]["GBQualifier_value"]
    
    #print(accession_number)
    #print(organism)
    #print(strain)
    #print(segment)
    #print(chromosome)
    #print(sequence)
    #print("\n")
    
    temp_df = pd.DataFrame({'accession_num':accession_number,
                            'strain':strain,
                            'isolate':isolate,
                            'segment':segment, 
                            'chromosome':chromosome, 
                            'sequence_length':len(sequence),
                            'host':host,
                            'location':location,
                            'subtype':subtype,
                            'organism':organism,
                            'collection_date':date,
                            'sequence':sequence}, index=[gb_entry])
    
    df_Compiled_Data=df_Compiled_Data.append(temp_df)
            
    #print("\n")

print("Done!")

df_Compiled_Data.to_excel("/PATH/save_to_excel_file.xlsx")

339
Going to download record 1 to 20
Going to download record 21 to 40
Going to download record 41 to 60
Going to download record 61 to 80
Going to download record 81 to 100
Going to download record 101 to 120
Going to download record 121 to 140
Going to download record 141 to 160
Going to download record 161 to 180
Going to download record 181 to 200
Going to download record 201 to 220
Going to download record 221 to 240
Going to download record 241 to 260
Going to download record 261 to 280
Going to download record 281 to 300
Going to download record 301 to 320
Going to download record 321 to 339
339
Done!


In [4]:
#This script will parse the contents of all available hanta sequences and save a fasta file for hanta, L segments of a certain size.

import re
from collections import OrderedDict 
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

output_handle = open("/PATH/SNV_all_L.fasta", "w")

all_data_dictionary=OrderedDict()
#Assign contents of pandas df to a dictionary:
for index_df, row_df in df_Compiled_Data.iterrows():
    sequence_name = "|".join((row_df['accession_num'], row_df['strain'], row_df['isolate'], row_df['segment'], str(row_df['sequence_length']), row_df['host'], row_df['location'], row_df['subtype'], row_df['organism'], row_df['collection_date']))
    all_data_dictionary.update({sequence_name:row_df['sequence']})

#Need to access ith items in the ordered dictionary.  Turn them into a list.
all_data_dictionary_items=list(all_data_dictionary.items())

for index_df, row_df in df_Compiled_Data.iterrows():
    
    #If correct virus continue parsing data:
    if re.search('Sin Nombre orthohantavirus', row_df['organism'], re.IGNORECASE) is not None:
        
        #If correct virus and L segment and right size, write to fasta.
        if re.search("L", row_df['segment'], re.IGNORECASE) is not None and (int(row_df["sequence_length"]) > 6000):
            record=SeqRecord(Seq(all_data_dictionary_items[index_df][1]), id=all_data_dictionary_items[index_df][0], description="")
            #print(record)
            SeqIO.write(record, output_handle, 'fasta-2line')
        
        #If correct virus and no value in segment and right size, write to fasta.
        if re.search("L", row_df['segment'], re.IGNORECASE) is None and (int(row_df["sequence_length"]) > 6000):
            record=SeqRecord(Seq(all_data_dictionary_items[index_df][1]), id=all_data_dictionary_items[index_df][0], description="")
            #print(record)
            SeqIO.write(record, output_handle, 'fasta-2line')

output_handle.close()

print("All L's downloaded!")


All L's downloaded!


In [5]:
#This script will parse the contents of all available hanta sequences and save a fasta file for hanta, M segments of a certain size.

import re
from collections import OrderedDict 
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

output_handle = open("/PATH/SNV_all_M.fasta", "w")

all_data_dictionary=OrderedDict()
#Assign contents of pandas df to a dictionary:
for index_df, row_df in df_Compiled_Data.iterrows():
    sequence_name = "|".join((row_df['accession_num'], row_df['strain'], row_df['isolate'], row_df['segment'], str(row_df['sequence_length']), row_df['host'], row_df['location'], row_df['subtype'], row_df['organism'], row_df['collection_date']))
    all_data_dictionary.update({sequence_name:row_df['sequence']})

#Need to access ith items in the ordered dictionary.  Turn them into a list.
all_data_dictionary_items=list(all_data_dictionary.items())

for index_df, row_df in df_Compiled_Data.iterrows():
    if re.search('Sin Nombre orthohantavirus', row_df['organism'], re.IGNORECASE) is not None:
        
        if re.search("M", row_df['segment'], re.IGNORECASE) is not None and (int(row_df["sequence_length"]) > 3000):
            record=SeqRecord(Seq(all_data_dictionary_items[index_df][1]), id=all_data_dictionary_items[index_df][0], description="")
            #print(record)
            SeqIO.write(record, output_handle, 'fasta-2line')
            
        #If correct virus and no value in segment and right size, write to fasta.
        if re.search("M", row_df['segment'], re.IGNORECASE) is None and (int(row_df["sequence_length"]) > 3000) and (int(row_df["sequence_length"]) < 4000):
            record=SeqRecord(Seq(all_data_dictionary_items[index_df][1]), id=all_data_dictionary_items[index_df][0], description="")
            #print(record)
            SeqIO.write(record, output_handle, 'fasta-2line')

output_handle.close()

print("All M's downloaded!")


All M's downloaded!


In [6]:
#This script will parse the contents of all available Hanta sequences and save a fasta file for hanta, S segments of a certain size.

import re
from collections import OrderedDict 
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

output_handle = open("/PATH/SNV_all_S.fasta", "w")

all_data_dictionary=OrderedDict()
#Assign contents of pandas df to a dictionary:
for index_df, row_df in df_Compiled_Data.iterrows():
    sequence_name = "|".join((row_df['accession_num'], row_df['strain'], row_df['isolate'], row_df['segment'], str(row_df['sequence_length']), row_df['host'], row_df['location'], row_df['subtype'], row_df['organism'], row_df['collection_date']))
    all_data_dictionary.update({sequence_name:row_df['sequence']})

#Need to access ith items in the ordered dictionary.  Turn them into a list.
all_data_dictionary_items=list(all_data_dictionary.items())

for index_df, row_df in df_Compiled_Data.iterrows():
    if re.search('Sin Nombre orthohantavirus', row_df['organism'], re.IGNORECASE) is not None:
        
        if re.search("S", row_df['segment'], re.IGNORECASE) is not None and (int(row_df["sequence_length"]) > 1500):
            record=SeqRecord(Seq(all_data_dictionary_items[index_df][1]), id=all_data_dictionary_items[index_df][0], description="")
            #print(record)
            SeqIO.write(record, output_handle, 'fasta-2line')
            
        #If correct virus and no value in segment and right size, write to fasta.
        if re.search("S", row_df['segment'], re.IGNORECASE) is None and (int(row_df["sequence_length"]) > 1500) and (int(row_df["sequence_length"]) < 2200):
            record=SeqRecord(Seq(all_data_dictionary_items[index_df][1]), id=all_data_dictionary_items[index_df][0], description="")
            #print(record)
            SeqIO.write(record, output_handle, 'fasta-2line')


output_handle.close()

print("All S's downloaded!")

All S's downloaded!


In [2]:
from Bio import Align
print(Align.__file__)


/opt/conda/lib/python3.7/site-packages/Bio/Align/__init__.py


In [1]:
from Bio import AlignIO
 
input_handle = open("/PATH/SNV_all.fasta", "r")
output_handle = open("/PATH/SNV_all.phy", "w")
 
alignments = AlignIO.parse(input_handle, "fasta")
AlignIO.write(alignments, output_handle, "phylip-relaxed")

 
output_handle.close()
input_handle.close()
print("Done!")

Done!
